In [1]:
"""# 🚀 Deployment & Demo Setup

This notebook prepares the Streamlit demo and deployment artifacts, writes a production-ready `streamlit_app.py` skeleton, an `agent_fsm.py` module, a pinned `requirements.txt`, and checks model artifact sizes.

Outputs:
- `streamlit_app.py`
- `agent_fsm.py`
- `requirements.txt`
- `deploy_instructions.md`
"""

'# 🚀 Deployment & Demo Setup\n\nThis notebook prepares the Streamlit demo and deployment artifacts, writes a production-ready `streamlit_app.py` skeleton, an `agent_fsm.py` module, a pinned `requirements.txt`, and checks model artifact sizes.\n\nOutputs:\n- `streamlit_app.py`\n- `agent_fsm.py`\n- `requirements.txt`\n- `deploy_instructions.md`\n'

In [2]:
import os, json, shutil
from pathlib import Path
import joblib

Path("artifacts").mkdir(exist_ok=True)
Path("artifacts/models").mkdir(parents=True, exist_ok=True)
Path("artifacts/eval").mkdir(parents=True, exist_ok=True)
Path("app").mkdir(exist_ok=True)
print("Directories verified.")


Directories verified.


In [3]:
def human_size(path):
    sz = os.path.getsize(path)
    for unit in ['B','KB','MB','GB']:
        if sz < 1024:
            return f"{sz:.1f}{unit}"
        sz /= 1024
    return f"{sz:.1f}TB"

model_files = list(Path("artifacts/models").glob("*"))
model_info = {str(p): human_size(str(p)) for p in model_files}
print("Model artifacts found:")
print(json.dumps(model_info, indent=2))


Model artifacts found:
{
  "artifacts\\models\\flow_iforest_20251108_231711.joblib": "277.0KB",
  "artifacts\\models\\flow_logreg_20251108_231711.joblib": "927.0B",
  "artifacts\\models\\flow_rf_20251108_231711.joblib": "50.6KB",
  "artifacts\\models\\packet_cnn_torch_v1.onnx": "23.9KB",
  "artifacts\\models\\packet_cnn_torch_v1.onnx.data": "4.5KB",
  "artifacts\\models\\packet_cnn_torch_v1.onnx.tmp": "19.6KB",
  "artifacts\\models\\packet_cnn_torch_v1.onnx.tmp.data": "4.5KB",
  "artifacts\\models\\packet_cnn_torch_v1.pt": "8.9KB"
}


In [4]:
"""## Requirements
We pin minimal CPU-only dependencies for Streamlit Cloud and local runs.
- `scikit-learn` for flow models
- `torch` (cpu) + `onnxruntime` for packet model inference
- `scapy` or `pyshark` for optional live capture (scapy recommended)
- `streamlit` for demo UI
- `matplotlib`, `pandas`, `numpy`, `joblib`, `tqdm`
"""

'## Requirements\nWe pin minimal CPU-only dependencies for Streamlit Cloud and local runs.\n- `scikit-learn` for flow models\n- `torch` (cpu) + `onnxruntime` for packet model inference\n- `scapy` or `pyshark` for optional live capture (scapy recommended)\n- `streamlit` for demo UI\n- `matplotlib`, `pandas`, `numpy`, `joblib`, `tqdm`\n'

In [5]:
reqs = [
    "python==3.10.*",
    "streamlit>=1.18.0",
    "scikit-learn>=1.2.0",
    "numpy>=1.24.0",
    "pandas>=1.5.0",
    "matplotlib>=3.6.0",
    "joblib>=1.2.0",
    "onnxruntime>=1.15.0",
    "torch==2.0.1+cpu",  # optional CPU wheel; adjust per environment
    "scapy>=2.5.0",
    "tqdm>=4.64.0"
]
with open("requirements.txt", "w") as f:
    f.write("\n".join(reqs))
print("✅ requirements.txt written")


✅ requirements.txt written


In [6]:
agent_code = r'''
# agent_fsm.py — deterministic FSM module for demo
import json
from datetime import datetime

class AgentFSM:
    def __init__(self, config=None):
        self.config = config or {
            "prob_threshold_investigate": 0.7,
            "prob_threshold_report": 0.85,
            "prob_threshold_contain": 0.9,
            "persistence_window": 2,
            "containment_duration": 3
        }
        self.state = "Monitoring"
        self.memory = []
        self.containment_timer = 0
        self.trace = []

    def _log(self, alert, action, reason):
        entry = {
            "time": datetime.now().isoformat(),
            "state": self.state,
            "packet_idx": int(alert.get("packet_idx", -1)),
            "prob": float(alert.get("packet_base_prob", alert.get("prob",0.0))),
            "action": action,
            "reason": reason
        }
        self.trace.append(entry)
        return entry

    def transition(self, new_state, reason):
        prev = self.state
        self.state = new_state
        return f"{prev} -> {new_state} ({reason})"

    def handle_alert(self, alert):
        prob = float(alert.get("packet_base_prob", alert.get("prob",0.0)))
        self.memory.append({"prob": prob, "packet_idx": alert.get("packet_idx")})
        self.memory = self.memory[-self.config["persistence_window"]:]
        if self.state == "Monitoring":
            if prob >= self.config["prob_threshold_investigate"]:
                msg = self.transition("Investigating", "High prob")
                return self._log(alert, "investigate", msg)
            else:
                return self._log(alert, "monitor", "below threshold")
        elif self.state == "Investigating":
            if all(m["prob"] > self.config["prob_threshold_report"] for m in self.memory):
                msg = self.transition("Reporting", "persistent")
                return self._log(alert, "report", msg)
            if prob < 0.5:
                msg = self.transition("Monitoring", "normalized")
                return self._log(alert, "monitor", msg)
            return self._log(alert, "investigate", "continue")
        elif self.state == "Reporting":
            if all(m["prob"] > self.config["prob_threshold_contain"] for m in self.memory):
                msg = self.transition("Containment", "critical")
                self.containment_timer = self.config["containment_duration"]
                return self._log(alert, "contain", msg)
            if prob < 0.6:
                msg = self.transition("Monitoring", "false alarm")
                return self._log(alert, "monitor", msg)
            return self._log(alert, "report", "persistence")
        elif self.state == "Containment":
            self.containment_timer -= 1
            if self.containment_timer <= 0:
                msg = self.transition("Monitoring", "release")
                return self._log(alert, "release", msg)
            return self._log(alert, "contain", "ongoing")

    def save_trace(self, path):
        with open(path, "w") as f:
            json.dump(self.trace, f, indent=2)
'''
with open("agent_fsm.py", "w") as f:
    f.write(agent_code)
print("✅ agent_fsm.py written")


✅ agent_fsm.py written


In [7]:
streamlit_code = r'''
# streamlit_app.py — demo app for CompactNetTrace
import streamlit as st
import joblib, json, os, time
import numpy as np
import onnxruntime as ort
from agent_fsm import AgentFSM

st.set_page_config(layout="wide", page_title="CompactNetTrace Demo")

st.title("CompactNetTrace — Lightweight NTA + Agentic AI Demo")

# Sidebar: model selection / demo controls
st.sidebar.header("Demo Controls")
use_sample = st.sidebar.checkbox("Use sample data (data/sample)", value=True)
stream_speed = st.sidebar.slider("Stream speed (s per alert)", 0.2, 2.0, 0.6)

# Load models
MODEL_DIR = "artifacts/models"
# find joblib flow model (pick latest)
flow_models = [f for f in os.listdir(MODEL_DIR) if f.startswith("flow_rf") or f.startswith("flow_logreg")]
flow_model = None
if flow_models:
    flow_model = joblib.load(os.path.join(MODEL_DIR, sorted(flow_models)[-1]))

onnx_path = os.path.join(MODEL_DIR, "packet_cnn_torch_v1.onnx")
onnx_sess = None
if os.path.exists(onnx_path):
    onnx_sess = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])
    input_name = onnx_sess.get_inputs()[0].name

st.sidebar.markdown("**Models loaded:**")
if flow_model: st.sidebar.write(f"- Flow model: {type(flow_model).__name__}")
if onnx_sess: st.sidebar.write("- Packet ONNX model loaded")

# Alerts source
ALERTS_PATH = "artifacts/eval/example_alerts.json"
if use_sample and os.path.exists(ALERTS_PATH):
    with open(ALERTS_PATH) as f:
        alerts = json.load(f)
else:
    alerts = []

# Agent
agent = AgentFSM()

# Main layout
col1, col2 = st.columns([2,1])

with col1:
    st.header("Agent Timeline")
    timeline_placeholder = st.empty()
with col2:
    st.header("Agent Log")
    log_placeholder = st.empty()

# Simulate streaming of alerts
for i, alert in enumerate(alerts):
    # Normalize keys for agent
    if "packet_base_prob" in alert:
        alert["prob"] = alert["packet_base_prob"]
    # agent handles alert
    log_entry = agent.handle_alert(alert)
    agent.save_trace("artifacts/agent_trace.json")

    # update agent log
    df = None
    try:
        with open("artifacts/agent_trace.json") as f:
            df = json.load(f)
    except:
        df = agent.trace
    log_placeholder.dataframe(df[-10:][::-1])

    # update timeline visualization (simple: use matplotlib via pyplot)
    import matplotlib.pyplot as plt
    from matplotlib.patches import Rectangle
    state_order = {"Monitoring":0,"Investigating":1,"Reporting":2,"Containment":3}
    state_colors = {"Monitoring":"#2ecc71","Investigating":"#f1c40f","Reporting":"#3498db","Containment":"#e74c3c"}
    trace_local = df
    times = list(range(len(trace_local)))
    states = [state_order[e["state"]] for e in trace_local]
    packets = [e.get("packet_idx") for e in trace_local]
    probs = [e.get("prob", 0.0) for e in trace_local]

    fig, ax = plt.subplots(figsize=(8,3))
    for st_name, y in state_order.items():
        ax.add_patch(Rectangle((-0.5, y - 0.5), len(trace_local) + 1, 1, color=state_colors[st_name], alpha=0.06))
    for j in range(len(trace_local)):
        color = list(state_colors.values())[states[j]]
        ax.scatter(j, states[j], color=color, edgecolor='black', s=80)
        if j>0:
            ax.plot([j-1,j],[states[j-1],states[j]], color=color, alpha=0.5)
        ax.text(j, states[j]+0.12, f"{packets[j]} ({probs[j]:.2f})", ha='center', fontsize=8)
    ax.set_yticks(list(state_order.values())); ax.set_yticklabels(list(state_order.keys()))
    ax.set_xlim(-0.5, max(5, len(trace_local)-1 + 0.5))
    ax.set_ylim(-0.5,3.5)
    ax.set_xlabel("Alert index")
    timeline_placeholder.pyplot(fig)

    time.sleep(stream_speed)

st.success("Stream finished (sample).")
'''
with open("streamlit_app.py", "w") as f:
    f.write(streamlit_code)
print("✅ streamlit_app.py written")


✅ streamlit_app.py written


In [8]:
deploy_md = r'''
# Deploying CompactNetTrace Demo (Streamlit Cloud)

1. Commit the repository to GitHub.
2. Ensure model artifacts are under `artifacts/models/` and the repo size < 100 MB. If models >100MB, use Git LFS or reduce size.
3. In Streamlit Cloud (https://share.streamlit.io), create a new app, connect your GitHub repo, and set the main file to `streamlit_app.py`.
4. Use the default settings (no secrets required).
5. If app fails due to dependency versions, adjust `requirements.txt` and redeploy.

Alternative: Hugging Face Spaces (Streamlit) — similar steps; ensure `runtime.txt` if needed.

Note: Streamlit Cloud ephemeral filesystem — model files must be in the repo or downloaded during startup. For large models, host them externally and download at app start.
'''
with open("deploy_instructions.md", "w") as f:
    f.write(deploy_md)
print("✅ deploy_instructions.md written")


✅ deploy_instructions.md written


In [9]:
"""## ✅ Deployment Checklist (produced)
- agent_fsm.py
- streamlit_app.py
- requirements.txt
- deploy_instructions.md

Next: push to GitHub and follow `deploy_instructions.md` to deploy on Streamlit Cloud.
"""

'## ✅ Deployment Checklist (produced)\n- agent_fsm.py\n- streamlit_app.py\n- requirements.txt\n- deploy_instructions.md\n\nNext: push to GitHub and follow `deploy_instructions.md` to deploy on Streamlit Cloud.\n'